## Ddip User Guide -- `%dipush` and `%dipull`

### Passing objects between the local notebook and the DDP process group


In [1]:
%load_ext Ddip
%makedip -r -g all --verbose False

Waiting for connection file: ~/.ipython/profile_default/security/ipcontroller-ippdpp_c-client.json


In [2]:
# Everything here is run in local notebook namespace.

import os
def var_printer(f): print(f"Process [{os.getpid()}] variable is :{f}")

foobar = { 1 : "foo", 2 : "bar" }
var_printer(foobar)

Process [8418] variable is :{1: 'foo', 2: 'bar'}


### `foobar` and `var_printer` are not visible in the DDP remote processes, unless we `%dipush` them over:

In [3]:
%%dip
# With %%dip at the top of a cell, everything here is executed in parallel in the DDP process group

import os
def var_printer(f): print(f"Process [{os.getpid()}] variable is :{f}")

var_printer(foobar)

RemoteError: NameError(name 'foobar' is not defined)

### From local to DDP:  `%dipush`

In [4]:
%dipush foobar

In [5]:
%%dip
var_printer(foobar)

Process [8472] variable is :{1: 'foo', 2: 'bar'}


### Let's create something remotely.  They are not visible in the local notebook.

In [6]:
%%dip --see all
# --see all is a flag to show outputs from all DDP processes, not only the rank-0 process.
remote_pid = os.getpid()
remote_pid

Out[0:3]: 8472

Out[1:3]: 8476

Out[2:3]: 8478

In [7]:
# back in local notebook cell without the %%dip at the top
remote_pid

NameError: name 'remote_pid' is not defined

### From remote DDP processes to local: `%dipull`

#### `%dipull` by default pulls in the object from RANK 0 process

In [9]:
%dipull remote_pid
print(f"Process [{os.getpid()}]: remote_pid is {remote_pid}")

Process [8418]: remote_pid is 8472


In [10]:
%dipull -r 1 remote_pid
print(f"Process [{os.getpid()}]: remote_pid is {remote_pid}")

Process [8418]: remote_pid is 8476


### It may seem tedious to create and remember what to push from one namespace to another.

Alternative, the cell magic `%%dip everywhere` dictates the same code to be run in all namespaces, thus creating/manipulating objects of the same name (but different instances underneath).  **To learn more about `%%dip everywhere`**, try [this notebook](usage_%%dip_%autodip.ipynb).

In [28]:
%%dip everywhere --see all
x = [ f"Different pid: {os.getpid()}", 2, ["a", "list"], {"and": "a dict"}]
x

['Different pid: 16359', 2, ['a', 'list'], {'and': 'a dict'}]

Out[0:10]: ['Different pid: 16413', 2, ['a', 'list'], {'and': 'a dict'}]

Out[1:10]: ['Different pid: 16417', 2, ['a', 'list'], {'and': 'a dict'}]

Out[2:10]: ['Different pid: 16418', 2, ['a', 'list'], {'and': 'a dict'}]